# 02-RDFy the data

This notebook explores the supplementary materials from the ACS Nano Paper:
>Labouta HI, Asgarian N, Rinker K, Cramb DT. Meta-Analysis of Nanoparticle Cytotoxicity via Data-Mining the Literature. ACS Nano. 2019 Jan 31; doi:10.1021/acsnano.8b07562 (Scholia)

ACS seems to block scrapers, so the supplementary data needs to be manually downloaded from the [supporting information link](https://pubs.acs.org/doi/suppl/10.1021/acsnano.8b07562/suppl_file/nn8b07562_si_001.xlsx) in the [ACS Nano page](https://pubs.acs.org/doi/full/10.1021/acsnano.8b07562), and then stored under [../data](../data).

To obtain the pickle file loaded at the beginning of the notebook, run [01-qc.ipynb](01-qc.ipynb).

## 1. Imports and functions used throughout the notebook

In [8]:
import pandas as pd
import numpy as np
import math
import os
import sys
from IPython.display import Markdown, display
from code import interact
import re
import rdflib
import requests

## 2. Loading data
Loads the pickled `Pandas DataFrame` obtained from [01-qc.ipynb](01-qc.ipynb).

In [5]:
file = "../data/nn9b07562_si_001.pkl"
df = pd.read_pickle(file)

## 3. Mapping terms with the eNanoMapper ontology
The [Ontology Lookup Service](https://www.ebi.ac.uk/ols/index) [search API](https://www.ebi.ac.uk/ols/docs/api) is used to programatically map terms and values in the data frame to eNanoMapper terms to create the subject, predicate, object triples.

### Predicates (column names)
Which terms can be mapped automatically?

In [35]:
base_url = "https://www.ebi.ac.uk"
get_query = "/ols/api/search?q={}&groupField=iri&start=0&ontology=enm"
for i in df.columns:
    r = requests.get(base_url+get_query.format(i))
    d = dict(r.json())
    print(i)
    for j in range(len(d["response"]["docs"])):
        print("\n\t", d["response"]["docs"][j]["iri"], " | ", d["response"]["docs"][0]["label"])

Nanoparticle

	 http://purl.bioontology.org/ontology/npo#NPO_707  |  nanoparticle

	 http://purl.enanomapper.org/onto/ENM_9000200  |  nanoparticle

	 http://purl.enanomapper.org/onto/ENM_9000245  |  nanoparticle

	 http://purl.enanomapper.org/onto/ENM_9000244  |  nanoparticle

	 http://purl.enanomapper.org/onto/ENM_9000249  |  nanoparticle

	 http://purl.enanomapper.org/onto/ENM_9000248  |  nanoparticle

	 http://purl.enanomapper.org/onto/ENM_9000247  |  nanoparticle

	 http://purl.enanomapper.org/onto/ENM_9000253  |  nanoparticle

	 http://purl.enanomapper.org/onto/ENM_9000014  |  nanoparticle

	 http://purl.enanomapper.org/onto/ENM_9000019  |  nanoparticle
Type: Organic (O)/inorganic (I)

	 http://purl.obolibrary.org/obo/OAE_0001603  |  Mobitz type I block AE

	 http://purl.bioontology.org/ontology/npo#NPO_802  |  Mobitz type I block AE

	 http://purl.obolibrary.org/obo/OAE_0003752  |  Mobitz type I block AE

	 http://www.ebi.ac.uk/efo/EFO_0011036  |  Mobitz type I block AE

	 http:/

This points at the lack of useful synonyms in the eNanoMapper ontology. Many of the terms will need to be matched manually.